Imported Libaries

In [1]:
import os
from sqlalchemy import create_engine, inspect

import inspect
from IPython.display import display as original_display

import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.data.path.append('C:/Users/jf79')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import spacy
from spellchecker import SpellChecker
from wordsegment import load, segment
from collections import Counter
from wordcloud import WordCloud

# Initialize external tools
spell = SpellChecker()
load()  # Load data for wordsegment
lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

vocabulary = [
    'FOI'
]
custom_stopwords = {
    'freedom','information','act','following','report',
    'please','dear','regard','yes','request','provide',
    'hammersmith','fulham','like','email','requesting',
    'see','attach'
}
for word in vocabulary:
    # Add custom vocabulary to SpellChecker
    spell.word_frequency.add(word)
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jf79\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jf79\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jf79\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Standard Functions

In [3]:
# Standard Functions #

# Function to clean labels in any plot functions
def clean_label(label):
    return label.replace('_', ' ').title()
 
# Function for getting the name of a Dataframe
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
 
# Function to validate the data in a Dataframe
def validate_data(df, show_counts=True):
    df_name = get_var_name(df)
    print(f'#########################################################################################################################################################################################\nDataFrame: {df_name}')
    #snapshot the dataset
    display(df)
    #check for unique values
    unique_counts = pd.DataFrame(df.nunique())
    unique_counts = unique_counts.reset_index().rename(columns={0:'No. of Unique Values', 'index':'Field Name'})
    print("Unique values per field:")
    pd.set_option('display.max_rows', None)
    display(unique_counts)
    pd.reset_option('display.max_rows')
    #checking for duplicates
    duplicate_count = df.duplicated().sum()
    print("\nNumber of duplicate rows:")
    print(duplicate_count,'\n')
    info = df.info(show_counts=show_counts)
    display(info)
    #summary stats
    print("\nSummary statistics:")
    display(df.describe())
    print('End of data validation\n#########################################################################################################################################################################################\n')
 
# Function to provide list for data sources as a dataframe when conducting analysis
def header_list(df):
    df_list_ = df.copy()
    df_list = df_list_.columns.tolist()
    df_list = pd.DataFrame(df_list)
    new_header = df_list.iloc[0]  # Get the first row for the header
    df_list = df_list[1:]  # Take the data less the header row
    df_list.columns = new_header  # Set the header row as the df header
    df_list.reset_index(drop=True, inplace=True)  # Reset index
   
    return df_list
 
def query_data(schema, data):
    # Define the SQL query
    query = f'SELECT * FROM [{schema}].[{data}]'
 
    # Load data into DataFrame
    df = pd.read_sql(query, engine)
 
    print(f'Successfully imported {data}')
    # Display the DataFrame
    return df

def display(df):
    # Attempt to get the name of the DataFrame from the caller's local variables
    frame = inspect.currentframe().f_back
    # Attempt to find the variable name corresponding to the DataFrame
    name = "Unnamed DataFrame"
    for var_name, var_value in frame.f_locals.items():
        if var_value is df:
            name = var_name
            break
 
    # If the name is not in the list to be excluded, print it
    if name not in {'df', 'Unnamed DataFrame', 'unique_counts'}:
        print(f"DataFrame: {name}")
    # Always display the DataFrame regardless of the name
    original_display(df)

def unique_values(df, display_df=True):
    # Extract unique values for each field and store them in a dictionary
    unique_values = {col: df[col].unique() for col in df.columns}
    # Find the maximum number of unique values
    max_length = max(len(values) for values in unique_values.values())
    # Create a dictionary for the new DataFrame with padded None values
    unique_df_data = {}
    for col, values in unique_values.items():
        unique_df_data[col] = list(values) + [None] * (max_length - len(values))
    # Create the new DataFrame
    unique_df = pd.DataFrame(unique_df_data)
    if display_df == True:
        # Set display options to show all rows and display the DataFrame
        pd.set_option('display.max_rows', None)
        display(unique_df.head(100))
        # Reset display options back to default
        pd.reset_option('display.max_rows')

def read_directory():
    directory = os.getcwd()
    files = os.listdir(os.getcwd())
    print(f"Your Current Directory is: {directory}")
    print("Files in: %s" % (files))

def export_to_csv(df):
    df_name = get_var_name(df)
    # Specify the directory and filename
    directory = r"C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry"
    file_path = f'{directory}\\{df_name}.csv'
    # Export the DataFrame to the specified directory
    df.to_csv(file_path, index=False)
    print(f'Successfully exported {df_name} to CSV')

Script Specific Functions

In [4]:
# Script Specific Functions #

def get_pos_tags(text):
    doc = nlp(text)
    return {token.text: token.tag_ for token in doc}

def is_valid_correction(original, pos_tags):
    # Skip correction if the original word matches these patterns
    if re.match(r"\d+(st|nd|rd|th)", original):  # Ordinal numbers
        return False
    if pos_tags and pos_tags.get(original) == 'NNP':
        return False
    return True

def is_valid_token(word):
    if len(word) > 2:
        return True
    if any(char.isdigit() for char in word):
        return True
    return False

def process_text(text):
    if not isinstance(text, str):  # Convert non-string types to string
        text = str(text)
    if not text.strip():  # Skip empty or whitespace strings
        return ""
    
    try:
        # Lowercase and clean text
        text = text.lower()
        text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
        text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\b', '', text)  # Remove email addresses
        text = re.sub(r'\s+', ' ', text)  # Normalize spaces after removal
        text = re.sub(r'[^\w\s]', '', text) # Remove puntuctation
        tokens = text.split()

        # Correct spelling and segment merged words
        corrected_tokens = []
        for word in tokens:
            split_words = segment(word)  # Split merged words
            for subword in split_words:
                corrected_word = spell.correction(subword) or word  # Correct spelling
                pos_tags = get_pos_tags(subword)
                if not is_valid_correction(subword, pos_tags):  # Skip invalid corrections
                    corrected_word = subword
                subword = corrected_word
            corrected_tokens.extend(split_words)
        
        # Remove single letter words
        corrected_tokens = [word for word in corrected_tokens if is_valid_token(word)]

        # Lemmatize and remove stop words
        tokens = [
            lemmatizer.lemmatize(word)
            for word in corrected_tokens
            if word not in stop_words
        ]
        return ' '.join(tokens)

    except Exception as e:
        print(f"Error processing text: {repr(text)}")
        raise e

# Tokenization function
def safe_tokenize(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return []

Database and CWD setup and connection

In [5]:
# Database credentials
db_host = 'LBHHLWSQL0001.lbhf.gov.uk'
db_port = '1433'
db_name = 'IA_ODS'
 
# Create the connection string for SQL Server using pyodbc with Windows Authentication
connection_string = f'mssql+pyodbc://@{db_host}:{db_port}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'

# Create the database engine
engine = create_engine(connection_string)

# Define the current working directory
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Fuel Poverty Analysis\Fuel Poverty General\Fuel Bible'
os.chdir(cwd)
files = os.listdir(os.getcwd())
print("Files in %r: %s" % (cwd, files))

Files in 'C:\\Users\\jf79\\OneDrive - Office Shared Service\\Documents\\H&F Analysis\\Fuel Poverty Analysis\\Fuel Poverty General\\Fuel Bible': ['certificates.csv', 'columns.csv', 'LICENCE.txt', 'recommendations.csv', 'schema.json', 'UPRN_to_LSOA_HF.csv']


In [6]:
# Read the CSV file into a DataFrame
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\FOI Sentiment Analysis\Data'
os.chdir(cwd)
read_directory()
columns = [
    'ClosedIRKey','CaseType','Division','Service','DateReceived','DateClosed','Due',
    'Status','Directorate','Team','Currentstage','Outcome','Outcomedate','ContactMethod',
    'Rating','DateAccepted','AssignedTo','Title','FirstName','LastName','Name','Address',
    'Town','County','Postcode','Details','TimescaleExtensionReason','ExtensionReason',
    'Extensionpermittedbylegislation','Representative'
]
FOI_details_data = pd.read_csv('FOI Details Data.csv', usecols=columns)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\FOI Sentiment Analysis\Data
Files in: ['FOI Details Data.csv']


In [7]:
FOI_details = FOI_details_data.copy()

# Data transformations
columns = [
    'Title','FirstName','LastName','Name',
    'Address','Town','County','Postcode'
]
FOI_address_details = pd.concat(
    (FOI_details[['ClosedIRKey']], FOI_details[columns]),
    axis=1
)
FOI_details = FOI_details.drop(columns=columns)
FOI_details['Details'] = FOI_details['Details'].fillna('').astype(str)
FOI_details.rename(columns={'Details':'details'}, inplace=True)

In [8]:
# Data Transformation for Sentiment Analysis
FOI_request = FOI_details[['details']].copy()

# Apply processing to 'details' and 'summary' columns
FOI_request['details'] = FOI_request['details'].apply(process_text)

# Tokenize 'details' and 'summary' columns
FOI_request['tokens'] = pd.DataFrame(FOI_request['details'].apply(safe_tokenize))

In [9]:
# Vectorise text
vectorizer = CountVectorizer(max_features=100, stop_words='english')
X = vectorizer.fit_transform(FOI_request['details'])

# Fit LDA model
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

for idx, topic in enumerate(lda.components_):
    print(f"Topic {idx}: ", [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:][::-1]])

# Example for Topic 0
for i in range(0,4):
    print(i)
    topic_idx = i
    word_contributions = {vectorizer.get_feature_names_out()[i]: lda.components_[topic_idx][i]
                        for i in range(len(vectorizer.get_feature_names_out()))}
    sorted_contributions = sorted(word_contributions.items(), key=lambda x: x[1], reverse=True)
    
    # Display top 10 contributing words
    for word, score in sorted_contributions[:10]:
        print(f"{word}: {score:.2f}")

# Assuming `corpus` is your preprocessed list of tokenized documents
# Create a dictionary from the corpus
id2word = Dictionary(FOI_request['tokens'])
 
# Create a bag-of-words corpus from the corpus
bow_corpus = [id2word.doc2bow(text) for text in FOI_request['tokens']]
 
coherence_scores = []
perplexities = []
 
topic_numbers = range(1, 6)
 
for n in topic_numbers:
    # Sklearn LDA model
    lda = LatentDirichletAllocation(n_components=n, random_state=42)
    lda.fit(X)  # X should be a term-frequency matrix (e.g., from CountVectorizer or TfidfVectorizer)
    # Perplexity for sklearn LDA
    perplexities.append(lda.perplexity(X))
    # Gensim LDA model
    lda_gensim = LdaModel(corpus=bow_corpus, id2word=id2word, num_topics=n, random_state=42)
    # Coherence using gensim's LDA model
    coherence_model = CoherenceModel(model=lda_gensim, texts=FOI_request['tokens'], dictionary=id2word, coherence='c_v')
    coherence_scores.append(coherence_model.get_coherence())
 
# Plotting results
plt.figure(figsize=(10, 6))
plt.plot(topic_numbers, coherence_scores, label='Coherence Score (C_v)', marker='o')
plt.plot(topic_numbers, perplexities, label='Perplexity', marker='x')
plt.xlabel('Number of Topics')
plt.ylabel('Score')
plt.title('Coherence and Perplexity for Topic Validation')
plt.legend()
plt.show()

Topic 0:  ['authority', 'local', 'school', 'year', 'question', 'number', 'child', 'area', 'total', 'data']
Topic 1:  ['contract', 'contact', 'date', 'address', 'copy', 'data', 'record', 'london', 'service', 'fund']
Topic 2:  ['council', 'property', 'road', 'housing', 'list', 'business', 'notice', 'rate', 'address', 'scheme']
Topic 3:  ['care', 'service', 'child', 'social', 'home', 'childrens', 'provider', 'adult', 'number', 'support']
Topic 4:  ['year', 'council', 'number', 'total', '2022', '2021', '2023', 'housing', 'financial', 'application']
0
authority: 2034.28
local: 1855.76
school: 1340.20
year: 927.04
question: 503.66
number: 437.44
child: 414.24
area: 412.10
total: 320.21
data: 272.54
1
contract: 1326.09
contact: 982.46
date: 953.51
address: 839.48
copy: 839.26
data: 822.71
record: 685.04
london: 582.73
service: 567.16
fund: 539.18
2
council: 1575.13
property: 1517.04
road: 846.32
housing: 525.04
list: 518.19
business: 512.49
notice: 512.41
rate: 502.07
address: 495.56
scheme: 

NameError: name 'Dictionary' is not defined

In [ ]:
# Flatten the tokenized details
all_tokens = [token for sublist in FOI_request['tokens'] for token in sublist]
# Count word frequencies
word_counts = Counter(all_tokens)
# Print the top 10 most common words
word_counts = pd.DataFrame([word_counts])
word_counts.reset_index(inplace=True)
word_counts = word_counts.melt(id_vars='index',var_name='Word',value_name='Count')
word_counts.sort_values(by='Count',ascending=False, inplace=True)
word_counts.reset_index(inplace=True)
word_counts[word_counts['Count'] > 100]